In [4]:
import numpy as np
from plyfile import PlyData

# Define SH constants
SH_C0 = 0.28209479177387814
SH_C1 = 0.4886025119029199
SH_C2 = np.array([
    1.0925484305920792,
    -1.0925484305920792,
    0.31539156525252005,
    -1.0925484305920792,
    0.5462742152960396
])
SH_C3 = np.array([
    -0.5900435899266435,
    2.890611442640554,
    -0.4570457994644658,
    0.3731763325901154,
    -0.4570457994644658,
    1.445305721320277,
    -0.5900435899266435
])

def get_sh_vec3(sh_coeffs, gaussianIndex, coeffIndex):
    # Assuming 16 coefficients per gaussian
    return sh_coeffs[gaussianIndex, coeffIndex, :]

def compute_sh(sh_coeffs, pos, gaussianIndex, camPos):
    # pos and camPos are numpy arrays of shape (3,)
    ray_direction = pos - camPos
    norm = np.linalg.norm(ray_direction)
    if norm == 0:
        return np.zeros(3)
    ray_direction /= norm
    x, y, z = ray_direction

    c = SH_C0 * get_sh_vec3(sh_coeffs, gaussianIndex, 0)

    c -= SH_C1 * get_sh_vec3(sh_coeffs, gaussianIndex, 1) * y
    c += SH_C1 * get_sh_vec3(sh_coeffs, gaussianIndex, 2) * z
    c -= SH_C1 * get_sh_vec3(sh_coeffs, gaussianIndex, 3) * x

    c += SH_C2[0] * get_sh_vec3(sh_coeffs, gaussianIndex, 4) * x * y
    c += SH_C2[1] * get_sh_vec3(sh_coeffs, gaussianIndex, 5) * y * z
    c += SH_C2[2] * get_sh_vec3(sh_coeffs, gaussianIndex, 6) * (2.0 * z * z - x * x - y * y)
    c += SH_C2[3] * get_sh_vec3(sh_coeffs, gaussianIndex, 7) * z * x
    c += SH_C2[4] * get_sh_vec3(sh_coeffs, gaussianIndex, 8) * (x * x - y * y)

    c += SH_C3[0] * get_sh_vec3(sh_coeffs, gaussianIndex, 9) * (3.0 * x * x - y * y) * y
    c += SH_C3[1] * get_sh_vec3(sh_coeffs, gaussianIndex, 10) * x * y * z
    c += SH_C3[2] * get_sh_vec3(sh_coeffs, gaussianIndex, 11) * (4.0 * z * z - x * x - y * y) * y
    c += SH_C3[3] * get_sh_vec3(sh_coeffs, gaussianIndex, 12) * z * (2.0 * z * z - 3.0 * x * x - 3.0 * y * y)
    c += SH_C3[4] * get_sh_vec3(sh_coeffs, gaussianIndex, 13) * x * (4.0 * z * z - x * x - y * y)
    c += SH_C3[5] * get_sh_vec3(sh_coeffs, gaussianIndex, 14) * (x * x - y * y) * z
    c += SH_C3[6] * get_sh_vec3(sh_coeffs, gaussianIndex, 15) * x * (x * x - 3.0 * y * y)

    c += 0.5

    # Clamp negative values to 0
    c = np.maximum(c, 0.0)
    return c

def compute_sh_for_vertex(input_ply_path, i, camPos):
    # Read the PLY file
    ply_data = PlyData.read(input_ply_path)
    vertex_data = ply_data['vertex'].data

    num_vertices = len(vertex_data)
    if i < 0 or i >= num_vertices:
        raise ValueError(f"Index i={i} is out of valid range. Total vertices: {num_vertices}")

    print("Total number of vertices:", num_vertices)

    # Extract the position of the i-th vertex
    pos_i = np.array([vertex_data['x'][i], vertex_data['y'][i], vertex_data['z'][i]])
    print(f"Position of vertex {i}:", pos_i)

    # Extract SH coefficients for the i-th vertex
    coeffs_i = np.zeros((16, 3), dtype=np.float32)

    # First coefficient: f_dc_0, f_dc_1, f_dc_2
    coeffs_i[0, :] = [vertex_data['f_dc_0'][i], vertex_data['f_dc_1'][i], vertex_data['f_dc_2'][i]]

    # Next 15 coefficients: f_rest_0 ~ f_rest_44 (15*3 = 45 values)
    for k in range(1, 16):
        start = (k - 1) * 3
        coeffs_i[k, 0] = vertex_data[f'f_rest_{start}'][i]
        coeffs_i[k, 1] = vertex_data[f'f_rest_{start+1}'][i]
        coeffs_i[k, 2] = vertex_data[f'f_rest_{start+2}'][i]
    print(f"SH coefficients for vertex {i} (coeffs_i):\n", coeffs_i)

    # Create SH coefficients array for a single vertex
    sh_coeffs = np.zeros((1, 16, 3), dtype=np.float32)
    sh_coeffs[0, :, :] = coeffs_i

    # Compute SH color for the vertex
    color_i = compute_sh(sh_coeffs, pos_i, 0, camPos)
    print(f"SH color for gaussian {i}:", color_i)

input_ply_path = './ply_files/point_cloud.ply'
camPos = np.array([0.0, 0.0, 10.0])
compute_sh_for_vertex(input_ply_path, 0, camPos)


Total number of vertices: 998988
Position of vertex 0: [ 0.58030313 -3.6833935   3.4494572 ]
SH coefficients for vertex 0 (coeffs_i):
 [[ 2.20295286e+00  1.60471892e+00  1.68436050e+00]
 [ 4.68679368e-02  7.08461925e-02  4.80314121e-02]
 [ 5.16026989e-02  5.34793399e-02  5.78413643e-02]
 [ 6.23360500e-02  4.56322841e-02  7.81637896e-03]
 [-3.45991488e-04  3.67758758e-02  1.78163908e-02]
 [ 1.03551028e-02  1.46439737e-02  2.77608968e-02]
 [ 2.49070246e-02  4.01756242e-02  6.76377788e-02]
 [ 7.26372376e-02  4.03042138e-02  4.36357036e-02]
 [ 7.82927796e-02  2.53746640e-02 -1.98679622e-02]
 [ 1.80556532e-02  2.33557597e-02  1.52131375e-02]
 [ 2.50392761e-02 -1.12344259e-02  2.26495769e-02]
 [ 3.22681703e-02  5.25894500e-02  3.43003459e-02]
 [ 4.83671352e-02  3.82581577e-02  4.37217504e-02]
 [ 4.62262966e-02  3.05492207e-02 -1.38234366e-02]
 [-9.77227557e-03  2.40709558e-02  1.18135130e-02]
 [-1.11381263e-02 -1.56093398e-02  2.91849114e-03]]
SH color for gaussian 0: [1.1101916 0.9766044 0.